# Harmonic Analysis 

In [ ]:
import xarray as xr 
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel
from joblib import delayed as jb_delayed

Number of parallel cpus : 

In [ ]:
n_workers = 6

### Functions

Creating the dictionary of tidal components frequencies : 

In [ ]:
freq_tide=dict()
for name,period in zip(['M2','S2','N2','K1','O1','K2','P1','Q1','M4','MS4','nu2','mu2','sN2','lambda2'],
                       [12.42060121,12.,12.65834824,23.93446959,25.81934171,11.96723606,24.06588766,26.868350,6.210300601,6.103339275,12.62600509,12.8717576,12.90537297,12.22177348]):
    freq_tide[name] = 2*np.pi/(3600*period) 

In [ ]:
def compute_G(nt,dt,tcomp):
    
    gtime=np.arange(0,nt*dt,dt)
    G = np.empty((nt,2*len(tcomp)))
    #eta1=np.empty((2*len(tcomp), ny,nx))
    for k in range(len(tcomp)):
        w=freq_tide[tcomp[k]]
        G[:,2*k]=np.cos(w*gtime*86400)
        G[:,2*k+1]=np.sin(w*gtime*86400)
    return G

def analysis(array,G): # dt= 1./24 hours
    
    M = np.dot(np.linalg.inv(np.dot(G.T,G)) , G.T)

    eta1 = np.dot(M, array)
    sol = np.dot(G, eta1)
    return eta1, sol

def compute_harmonic_analysis(array_ssh,dt=1/24,tcomp=['M2']):
    nt = len(array_ssh)
    G = compute_G(nt,dt,tcomp)
    ssh_analysis = np.zeros_like(array_ssh)
    for i in range(array_ssh.shape[1]):
        for j in range(array_ssh.shape[2]):
            eta , ssh_analysis[:,i,j]=analysis(array_ssh[:,i,j],G)
    return ssh_analysis


### Loading SSH 

In [ ]:
import glob 
list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_it/hawaii/MITgcm_it_2012*.nc")
list_files.sort()
list_files.pop(0)

In [ ]:
ds = xr.open_mfdataset(list_files,engine='netcdf4')

In [ ]:
ssh = ds.ssh_it1

In [ ]:
array_ssh = ssh.values

### Processing

In [ ]:
ssh_coherent = compute_harmonic_analysis(array_ssh)

In [ ]:
ssh_coherent.shape

In [ ]:
91*24

In [ ]:
print(len(date_array))

In [ ]:
date_array = np.arange(np.datetime64("2012-05-02"),np.datetime64("2012-08-02"))

for i in range (len(date_array)-1) : 
    ds_it_coherent = ssh.sel(time=slice(date_array[i],date_array[i+1]-np.timedelta64(1,'h'))).copy(deep=True,data=ssh_coherent[i*24:(i+1)*24,:,:])
    ds_it_coherent = ds_it_coherent.rename("ssh_it1_coherent_analysis")
    ds_it_coherent.to_netcdf("/bettik/bellemva/MITgcm/MITgcm_it/hawaii/coherent_part/MITgcm_it_"+date_array[i].astype('str').replace('-','')+".nc")

    print(date_array[i])


In [ ]:
dates = np.arange(np.datetime64("2012-05-02"),np.datetime64("2012-08-01"))

# deleting the content of file.sh 
file = open("./nco_bash.sh","w")
file.close()

for d in dates:

    command = "ncks -h -A "

    command += "/bettik/bellemva/MITgcm/MITgcm_it/hawaii/coherent_part/MITgcm_it_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_it/hawaii/MITgcm_it_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("./nco_bash.sh","a")
    
    file.write(command)

    file.close()

#### Plotting 

In [ ]:
import xarray as xr 
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
ds = xr.open_mfdataset("/bettik/bellemva/MITgcm/MITgcm_it/hawaii/MITgcm_it_201206*.nc",parallel=True,drop_variables=["ssh_it1","ssh_it2", "ssh_it3","ssh_it_tot"])
ds.sel(longitude = slice(190,200),latitude = slice(20,30))

In [ ]:
ssh_it_coherent = ds.ssh_it1_coherent.values

In [ ]:
ssh_it_coherent_analysis = ds.ssh_it1_coherent_analysis.values

In [ ]:
def create_plot(i):
    vmin=-0.1
    vmax=0.1

    fig,ax = plt.subplots(1,3,figsize=(16,4))

    plot = ax[0].pcolormesh(ds.longitude,ds.latitude,ssh_it_coherent[i,:,:],vmin=vmin,vmax=vmax)
    ax[0].set_aspect("equal")
    ax[0].set_title("SSH coherent wt. Fourrier")
    plt.colorbar(plot,ax=ax[0])

    plot_analysis = ax[1].pcolormesh(ds.longitude,ds.latitude,
                                    ssh_it_coherent_analysis[i,:,:],vmin=vmin,vmax=vmax)
    ax[1].set_aspect("equal")
    ax[1].set_title("SSH coherent wt. Analysis")
    plt.colorbar(plot_analysis,ax=ax[1])

    plot_diff = ax[2].pcolormesh(ds.longitude,ds.latitude,
                                ssh_it_coherent_analysis[i,:,:]-ssh_it_coherent[i,:,:],cmap="RdBu",vmin=-0.06,vmax=0.06)
    ax[2].set_aspect("equal")
    ax[2].set_title("Difference")
    plt.colorbar(plot_diff,ax=ax[2])

    str_date = ds.time[i].values.astype("datetime64[h]").astype("str")
    plt.suptitle(str_date)
    plt.savefig(f"./frames/{str_date}.png")

    #ds.time[i].values.astype("datetime64[h]").astype("str")



In [ ]:
import matplotlib
matplotlib.use("Agg")

for i in range (ds.dims["time"]):
    create_plot(i)

In [ ]:
plt.plot(ssh_it_coherent[:,280,400])
plt.plot(ssh_it_coherent_analysis[:,280,400])
#plt.xlim(200,400)